### Mod 1 Project --- Data Movie

In [1]:
#top 50 lucrative movies from  tn_movie_budgets and bom_movie_gross   (find their ratings)
# most (top50) popular movies from tmdb_movies
# highest(top 50) ratings from imdb_title_ratings

# select common movies from above list
#find their genre, directors

### Import libraries & database

In [2]:
!pip install mysql-connector-python

In [3]:
import pandas as pd
import numpy as np
import mysql.connector
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')
cur = con.cursor()

# Data Cleaning

### IMDB Titles

In [5]:
query = ('''
SELECT *
FROM imdb_title_basics imdb_b
JOIN imdb_title_crew imdb_c USING (tconst)
JOIN imdb_title_ratings imdb_e USING (tconst)
''')
imdb_titles = pd.read_sql_query(query, con)
#joining all IMDB title tables using title ids and name ids

In [6]:
imdb_titles.isna().sum()
#checking for null values

tconst                 0
primary_title          0
original_title         0
start_year             0
runtime_minutes     7620
genres                 0
directors              0
writers            10561
averagerating          0
numvotes               0
dtype: int64

In [7]:
imdb_titles = imdb_titles.dropna(subset=['runtime_minutes'])
imdb_titles = imdb_titles.dropna(subset=['writers'])
#Removing rows with null values for runtime minutes and writers

In [8]:
pd.set_option('display.max_columns',None)
# Want to see all columns to see which are going to be useful

In [9]:
imdb_titles['genres'] = imdb_titles['genres'].map(lambda x: x.split(','))
imdb_titles['directors'] = imdb_titles['directors'].map(lambda x: x.split(','))
imdb_titles['writers'] = imdb_titles['writers'].map(lambda x: x.split(','))
# Converting genres, directors, writers to lists, so we can easily check most popular later.

In [10]:
imdb_titles['age_of_movie']= 2020 - imdb_titles['start_year']
#Creating new column to have "age of movie" as a reference

In [11]:
imdb_titles.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes,age_of_movie
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"[Action, Crime, Drama]",[nm0712540],"[nm0023551, nm1194313, nm0347899, nm1391276]",7.0,77,7
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,[Drama],[nm0000080],"[nm0000080, nm0462648]",6.9,4517,2
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"[Comedy, Drama, Fantasy]","[nm0765384, nm0749914]","[nm1360635, nm0749914]",6.5,119,3
6,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"[Adventure, Animation, Comedy]",[nm0365480],[nm0365480],8.1,263,3
7,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,[Drama],[nm1030585],"[nm1030585, nm1921430]",6.8,451,10


### IMDB Crew

In [12]:
query = ('''
SELECT *
FROM imdb_name_basics
''')
imdb_crew = pd.read_sql_query(query, con)

In [13]:
imdb_crew.isna().sum()
# Checking for null values

nconst                     0
primary_name               0
birth_year            523912
death_year            599865
primary_profession         0
known_for_titles           0
dtype: int64

In [14]:
imdb_crew = imdb_crew.fillna(0)
#Replacing null values with 0

In [15]:
imdb_crew['birth_year'] = imdb_crew['birth_year'].map(lambda x: int(x))
imdb_crew['death_year'] = imdb_crew['death_year'].map(lambda x: int(x))
#Converting birth years and death years to integer

In [16]:
imdb_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   nconst              606648 non-null  object
 1   primary_name        606648 non-null  object
 2   birth_year          606648 non-null  int64 
 3   death_year          606648 non-null  int64 
 4   primary_profession  606648 non-null  object
 5   known_for_titles    606648 non-null  object
dtypes: int64(2), object(4)
memory usage: 27.8+ MB


In [17]:
imdb_crew['primary_profession'] = imdb_crew['primary_profession'].map(lambda x: x.split(','))
imdb_crew['known_for_titles'] = imdb_crew['known_for_titles'].map(lambda x: x.split(','))
#Converting these to lists so we can easily reference them later

In [18]:
imdb_crew.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0038355, tt0117057, tt0071877, tt0037382]"
1,nm0000003,Brigitte Bardot,1934,0,"[actress, soundtrack, producer]","[tt0049189, tt0057345, tt0054452, tt0059956]"
2,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0069467, tt0050986, tt0083922, tt0050976]"
3,nm0000006,Ingrid Bergman,1915,1982,"[actress, soundtrack, producer]","[tt0071877, tt0036855, tt0038109, tt0038787]"
4,nm0000007,Humphrey Bogart,1899,1957,"[actor, soundtrack, producer]","[tt0034583, tt0033870, tt0037382, tt0043265]"


In [19]:
imdb_crew['age'] = imdb_crew.apply(lambda row: (2020 - row['birth_year']) if row['death_year'] == 0 else None, axis=1)
#Adding an age column. If the age is 2020, it means that there is no birth year available. If the age is equal to None, it means the person is dead.

### BOM

In [20]:
query = ('''
SELECT * FROM bom_movie_gross
''')
bom = pd.read_sql_query(query, con)

In [21]:
#drop Null value in domestic_gross
bom = bom.dropna(subset =['domestic_gross'])

#convert Null value in foreign_gross to 0
bom.foreign_gross = bom.foreign_gross.fillna(0)

#convert domestic_gross and foreign_gross from float to int
bom.domestic_gross = bom.domestic_gross.map(lambda x: int(x))   
bom.foreign_gross = bom.foreign_gross.map(lambda x: int(x))  

In [22]:
#remove spaces on both end of strings
bom.title = bom.title.map(lambda x: x.strip())
bom.studio = bom.studio.map(lambda x: x.strip())

### TN

In [23]:
query_tn = ('''
SELECT * FROM tn_movie_budgets
''')
tn = pd.read_sql_query(query_tn, con)

In [24]:
# convert release_date to date format
tn.release_date = tn.release_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y'))

# convert production_budget, domestic_gross, worldwide_gross to date format
tn.production_budget = tn.production_budget.map(lambda x: int(x.replace(',','').replace('$','')))
tn.domestic_gross = tn.domestic_gross.map(lambda x: int(x.replace(',','').replace('$','')))
tn.worldwide_gross = tn.worldwide_gross.map(lambda x: int(x.replace(',','').replace('$','')))

#add profit column
tn['profit'] = tn.worldwide_gross - tn.production_budget

In [25]:
#remove spaces on both end of strings
tn.movie = tn.movie.map(lambda x: x.strip())

### TMDB

In [26]:
query_tmdb = ('''
SELECT * FROM tmdb_movies
''')
tmdb = pd.read_sql_query(query_tmdb, con)

In [27]:
query_tmdb_link = ('''
SELECT * FROM tmdb_imdb_link
''')
tmdb_link = pd.read_sql_query(query_tmdb_link, con)

In [28]:
#convert genre_ids to int
tmdb.genre_ids.map(lambda x: int(x))

#remove spaces on both end of strings
tmdb.original_language = tmdb.original_language.map(lambda x: x.strip())
tmdb.original_title = tmdb.original_title.map(lambda x: x.strip())
tmdb.title = tmdb.title.map(lambda x: x.strip())

In [29]:
#Join tmdb and tmdb_link tables
tmdb = tmdb.join(tmdb_link.set_index('tmdb_id'), on = 'id', how = 'left')

In [30]:
#clean the joined table
tmdb.imdb_id = tmdb.imdb_id.fillna(0).map(lambda x: int(x))

### RT

In [31]:
query_rt_info = ('''
SELECT * FROM rt_movie_info
''')
rt_info = pd.read_sql_query(query_rt_info, con)

In [32]:
query_rt_reviews = ('''
SELECT * FROM rt_reviews
''')
rt_reviews = pd.read_sql_query(query_rt_reviews, con)

In [33]:
query_rt_link = ('''
SELECT * FROM rt_imdb_link
''')
rt_link = pd.read_sql_query(query_rt_link, con)

### RT_Info Cleaning

In [34]:
#replace empty value in theater_date and dvd_date with Jan 1,1000
rt_info['theater_date'].loc[rt_info.theater_date == ''] = 'Jan 1, 1000'
rt_info['dvd_date'].loc[rt_info.dvd_date == ''] = 'Jan 1, 1000'

C:\Users\yazan\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [35]:
#convert theater_date and dvd_date to datetime format
rt_info.theater_date = rt_info.theater_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y').date())
rt_info.dvd_date = rt_info.dvd_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y').date())

In [36]:
#clean box_office date
rt_info['box_office'].loc[rt_info.box_office == ''] = '0'
rt_info['box_office'] = rt_info.box_office.map(lambda x: int(x.replace(',', '')))

In [37]:
#clean runtime (replace empty value with 0)
rt_info['runtime'].loc[rt_info.runtime == ''] = '0'
rt_info['runtime'] = rt_info.runtime.map(lambda x: int(x.replace('minutes', '')))

In [38]:
#remove spaces on both end of strings
rt_info.synopsis = rt_info.synopsis.map(lambda x: x.strip())
rt_info.rating = rt_info.rating.map(lambda x: x.strip())
rt_info.genre = rt_info.genre.map(lambda x: x.strip())
rt_info.director = rt_info.director.map(lambda x: x.strip())
rt_info.writer = rt_info.writer.map(lambda x: x.strip())

In [39]:
rt_info['genre'] = rt_info['genre'].map(lambda x: x.split('|'))
rt_info['writer'] = rt_info['writer'].map(lambda x: x.split('|'))
rt_info['director'] = rt_info['director'].map(lambda x: x.split('|'))
#Converting genre, writer, director to list so we can reference them easily

In [40]:
rt_info = rt_info.drop(columns=['box_office','currency'],axis=1)
#Dropping box_office because we have better data on another table, dropping currency because we removed box office data

### RT_Reviews Cleaning

In [41]:
#convert date to datetime format
rt_reviews.date = rt_reviews.date.map(lambda x: dt.datetime.strptime(x, '%B %d, %Y').date())

In [42]:
#convert top_critic to int
rt_reviews.top_critic = rt_reviews.top_critic.map(lambda x: int(x))

In [43]:
#remove spaces on both end of strings
rt_reviews.review = rt_reviews.review.map(lambda x: x.strip())
rt_reviews.rating = rt_reviews.rating.map(lambda x: x.strip())
rt_reviews.fresh = rt_reviews.fresh.map(lambda x: x.strip())
rt_reviews.critic = rt_reviews.critic.map(lambda x: x.strip())
rt_reviews.publisher = rt_reviews.publisher.map(lambda x: x.strip())

### RT_link cleaning

In [44]:
rt_link.year = rt_link.year.fillna(1000)

In [45]:
rt_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17468 entries, 0 to 17467
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   imdbid       17468 non-null  int64  
 1   rtid         17468 non-null  int64  
 2   title        17468 non-null  object 
 3   year         17468 non-null  float64
 4   mpaa_rating  17468 non-null  object 
 5   runtime      17468 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 818.9+ KB


In [46]:
#replace empty space to year 1000 in year
rt_link.year = rt_link.year.fillna(1000)

#convert date to datetime format
rt_link.year = rt_link.year.map(lambda x: (int(x)))

In [47]:
#replace empty space in runtime to 0
rt_link['runtime'].loc[rt_link.runtime == ''] = '0'

#convert runtime to int format
rt_link.runtime = rt_link.runtime.map(lambda x: (int(x)))

# Top 200 Profitable Movies after 2000

### BOM Database

In [48]:
# get top 200 lucrative domestic movies
bom_top200_domestic = bom.sort_values('domestic_gross', ascending = False).head(200)   

### TN Database

In [49]:
# get top 200 revenue domestic movies
tn_top200_domestic = tn.sort_values('domestic_gross',ascending=False).head(200)

# get top 200 revenue worldwide movies
tn_top200_world = tn.sort_values('worldwide_gross',ascending=False).head(200)

# get top 200 profit movies
tn_top200_profit = tn.sort_values('profit',ascending=False).head(200)

# Top 200 Highest Rated Movies after 2000

In [50]:
#Because the ratings on the RT table are inconsistent (Some show A+, some 4/5, some 7.5/10, etc), I will use the IMDB and TMDB ratings

In [51]:
#To get tmdb['imdb_id'] to line up with the imdb table, I want to remove the "tt" at the beginning of the values in the imdb_titles['tconst'] column.
imdb_titles['tconst'] = imdb_titles['tconst'].str[2:]
#Similarly, I want to make sure that tmdb['imdb_id'] has 7 digits, using 0 as a filler, so that they match
tmdb['imdb_id'] = tmdb['imdb_id'].astype(str).str.rjust(7,'0')

In [52]:
rating = imdb_titles.merge(tmdb,left_on='tconst',right_on='imdb_id')

In [55]:
#Dropping some less relevant columns
rating = rating.drop(columns=['original_title_x','original_title_y','id','genre_ids'])

In [56]:
rating.head()

,tconst,primary_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes,age_of_movie,original_language,popularity,release_date,title,vote_average,vote_count,imdb_id
0,0069049,The Other Side of the Wind,2018,122.0,[Drama],[nm0000080],"[nm0000080, nm0462648]",6.9,4517,2,en,9.800,2018-11-02,The Other Side of the Wind,7.0,64,0069049
1,0249516,Foodfight!,2012,91.0,"[Action, Animation, Comedy]",[nm0440415],"[nm0440415, nm0923312, nm0295165, nm0841854, n...",1.9,8248,8,en,4.705,2013-05-07,Foodfight!,2.1,46,0249516
2,0293069,Dark Blood,2012,86.0,[Thriller],[nm0806293],[nm0059172],6.6,1053,8,en,1.721,2013-03-06,Dark Blood,6.1,7,0293069
3,0315642,Wazir,2016,103.0,"[Action, Crime, Drama]",[nm2349060],"[nm7430071, nm1249226, nm0006765, nm0430785, n...",7.1,15378,4,hi,3.881,2016-01-08,Wazir,6.6,63,0315642
4,0323808,The Wicker Tree,2011,96.0,"[Drama, Horror]",[nm0362736],[nm0362736],3.9,2328,9,en,2.962,2012-01-27,The Wicker Tree,4.4,25,0323808


In [58]:
rating['aggregate_score'] = round(rating.apply(lambda row: (row['averagerating'] * row['numvotes']) + (row['vote_average'] * row['vote_count']) * row['popularity'],axis=1),2)
#Creating an aggregate score: (IMDB Rating * IMDB number of votes) + (TMDB average rating * TMDB numvotes) * Popularity

In [59]:
top_200_rating = rating.sort_values(by=['aggregate_score'],ascending=False).head(200)

In [60]:
top_200_rating.head()

,tconst,primary_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes,age_of_movie,original_language,popularity,release_date,title,vote_average,vote_count,imdb_id,aggregate_score
849,1375666,Inception,2010,148.0,"[Action, Adventure, Sci-Fi]",[nm0634240],[nm0634240],8.8,1841066,10,en,27.920,2010-07-16,Inception,8.3,22186,1375666,21342675.70
153,0848228,The Avengers,2012,143.0,"[Action, Adventure, Sci-Fi]",[nm0923736],"[nm0923736, nm0672015]",8.1,1183655,8,en,50.289,2012-05-04,The Avengers,7.6,19673,0848228,17106555.28
130,0816692,Interstellar,2014,169.0,"[Adventure, Drama, Sci-Fi]",[nm0634240],"[nm0634300, nm0634240]",8.6,1299334,6,en,28.440,2014-11-05,Interstellar,8.2,18597,0816692,15511241.58
7139,4154756,Avengers: Infinity War,2018,149.0,"[Action, Adventure, Sci-Fi]","[nm0751577, nm0751648]","[nm1321655, nm1321656, nm0498278, nm0456158, n...",8.5,670926,2,en,80.773,2018-04-27,Avengers: Infinity War,8.3,13948,4154756,15053831.97
3206,2015381,Guardians of the Galaxy,2014,121.0,"[Action, Adventure, Comedy]",[nm0348181],"[nm3238648, nm0314153, nm0562554, nm0348181, n...",8.1,948394,6,en,49.606,2014-08-01,Guardians of the Galaxy,7.9,17958,2015381,14719505.33
